In [1]:
from nltk.corpus import wordnet,stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import WordNetLemmatizer
import string
import nltk
from sklearn.model_selection import GridSearchCV,cross_val_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
import seaborn as sns
sns.set()

[nltk_data] Downloading package punkt to /Users/deepak17/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
training_data = pd.read_csv("0000000000002747_training_twitter_x_y_train.csv")
test_text = pd.read_csv("0000000000002747_test_twitter_x_test.csv")['text']

In [3]:
texts = training_data['text']
Y_train = training_data['airline_sentiment']

In [4]:
X =[]
X_test = []
for text in texts:
    X.append(word_tokenize(text))

for text in test_text:
    X_test.append(word_tokenize(text))
    

In [5]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/deepak17/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
stopwords = stopwords.words('english') + list(string.punctuation)
lemmatizer = WordNetLemmatizer()
def clean_doc(doc):
    clean_words=[]
    for word in doc:
        if word.lower() not in stopwords and ( word.isalpha()):
            clean_words.append( lemmatizer.lemmatize( word , simple(pos_tag([word])[0][1])).lower())
    return clean_words

In [7]:
def simple(tag):
    if tag.startswith('N'):
        return wordnet.NOUN
    if tag.startswith('V'):
        return wordnet.VERB
    if tag.startswith('A'):
        return wordnet.ADV
    if tag.startswith('J'):
        return wordnet.ADJ
    else:
        return wordnet.NOUN

In [8]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/deepak17/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [9]:
import nltk
nltk.download('wordnet')
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/deepak17/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/deepak17/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [10]:
X_train= [ ' '.join( clean_doc(doc)) for doc in X ]
X_test= [ ' '.join( clean_doc(doc)) for doc in X_test ]
X_test[0]

'americanair car gng dfw pulled ago icy road aa since ca reach arpt wat'

In [11]:
Y_train = Y_train.replace({
    'negative' : 0,
    'neutral' : 1,
    'positive' : 2,
})
Y_train

/var/folders/3s/zxhyw_8n5m163lk9krg9v28r0000gp/T/ipykernel_11221/2840477981.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Y_train = Y_train.replace({


0        0
1        2
2        2
3        0
4        0
        ..
10975    1
10976    2
10977    0
10978    0
10979    0
Name: airline_sentiment, Length: 10980, dtype: int64

In [12]:
cv = CountVectorizer(max_features=3000)
cv.fit(X_train)

CountVectorizer(max_features=3000)

In [13]:
cv.get_feature_names_out()

array(['aa', 'aadvantage', 'aafail', ..., 'zkatcher', 'zone', 'zurich'],
      dtype=object)

In [14]:
X_train = cv.transform(X_train)
X_test = cv.transform(X_test)

In [15]:
X_train = X_train.todense()
X_test = X_test.todense()

In [16]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

X_train = np.asarray(X_train)
X_test = np.asarray(X_test)

model1 = MultinomialNB()
scores = cross_val_score(model1, X_train, Y_train)
print(scores.mean())

0.7645719489981786


In [17]:
from sklearn.naive_bayes import MultinomialNB
model1 = MultinomialNB()
cross_val_score(model1,X_train,Y_train).mean()

0.7645719489981786

In [18]:
from sklearn.linear_model import LogisticRegression
model2 = LogisticRegression(max_iter=2000,multi_class='ovr')
cross_val_score(model2,X_train,Y_train).mean()

0.7853369763205829

In [19]:
from sklearn.ensemble import RandomForestClassifier
model3 = RandomForestClassifier()
cross_val_score(model3,X_train,Y_train).mean()

0.7541894353369762

In [20]:
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
model = Sequential()
model.add(Dense(1024,activation='relu',input_dim = 3000))
model.add(Dense(512,activation='sigmoid'))
model.add(Dense(256,activation='relu'))
model.add(Dense(128,activation='sigmoid'))
model.add(Dense(64,activation='relu'))
model.add(Dense(3,activation='sigmoid'))

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [22]:
model.fit(X_train,  to_categorical(Y_train)  ,epochs=20,batch_size = 50)

Epoch 1/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.6473 - loss: 0.8277
Epoch 2/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.8349 - loss: 0.4411
Epoch 3/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9045 - loss: 0.2779
Epoch 4/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9478 - loss: 0.1660
Epoch 5/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9747 - loss: 0.0902
Epoch 6/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.9818 - loss: 0.0616
Epoch 7/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9895 - loss: 0.0405
Epoch 8/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.9887 - loss: 0.0394
Epoch 9/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9887 - loss: 0.0359
Epoch 10/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.9918 - loss: 0.0266
Epoch 11/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.9905 - loss: 0.0283
Epoch 12/20
220/220 ━━━━━━━━━━━━━━━━━━━━ 

In [23]:
model2 = LogisticRegression(max_iter=2000,multi_class='ovr')
model2.fit(X_train,Y_train)
predict  = model2.predict(X_test)
predict

array([0, 0, 0, ..., 0, 2, 1])

In [24]:
predict = pd.Series(predict).replace({
    0:'negative',
    1:'neutral',
    2:'positive'
})
predict

0       negative
1       negative
2       negative
3       negative
4       positive
          ...   
3655    negative
3656    negative
3657    negative
3658    positive
3659     neutral
Length: 3660, dtype: object

In [25]:
predict.to_csv('Prediction.csv',index=False,header=False)